In [ ]:
# %load simulations
from TreeClassifier import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import tree
import numpy as np
import datetime
import os

In [ ]:
depth = 10
output_dir = r"C:\Temp\simulations-tests"

In [ ]:
def get_simulation_data_1(d=100,n=1000):
    X = np.random.normal(size=[n, d])
    y = np.greater_equal(np.sum(X, axis=1), 0.0)
    return X, y


def get_simulation_data_2(d=100,n=1000):
    X = np.random.normal(size=[n, d])
    w = np.random.normal(size=[d, 1])
    y = np.greater_equal(np.dot(X,w).reshape(-1), 0.0)
    print 'positive class weight {}'.format(float(y.sum()) / len(y))
    return X, y


def get_simulation_data_3(d=100,n=1000):
    X = np.random.normal(size=[n, d])
    w1 = np.random.normal(size=[d, 1])
    w2 = np.random.normal(size=[d, 1])
    y = np.greater_equal(np.multiply(np.dot(X,w1),np.dot(X,w2)).reshape(-1), 0.0)
    print 'positive class weight {}'.format(float(y.sum()) / len(y))
    return X, y

def get_simulation_data_4(d=100,n=1000):
    X = np.random.normal(size=[n, d])
    X1 = X[:,:d/2]
    X2 = X[:,d/2:]
    w1 = np.random.normal(size=[d/2, 1])
    w2 = np.random.normal(size=[d/2, 1])
    y = np.greater_equal(np.multiply(np.dot(X1,w1),np.dot(X2,w2)).reshape(-1), 0.0)
    print 'positive class weight {}'.format(float(y.sum()) / len(y))
    return X, y

In [ ]:
def run_comparison(X,y):
    tree_classifier = TreeClassifier(0.001, depth, normalizer_mode="norm", feature_drop_probability=0.0)
    stochastic_scores = cross_val_score(tree_classifier, X, y, cv=5)
    print "stochastic"
    print stochastic_scores.mean()

    tree_scores = cross_val_score(tree.DecisionTreeClassifier(max_depth=depth), X, y, cv=5)
    print "regular"
    print tree_scores.mean()
    
    svm_scores = cross_val_score(LinearSVC(), X_n, y, cv=5)
    print "svm"
    print svm_scores.mean()
    
    return stochastic_scores, tree_scores, svm_scores

In [ ]:
def manage_run(data_generation, sim_name, noise = False):
    test_time = str(datetime.datetime.now()).replace(':','-').replace(' ','-').replace('.','-')
    name = sim_name
    if noise:
        name = name + 'noise-'
    name = name + test_time
    current_dir = os.path.join(output_dir,)
    if not os.path.exists(current_dir):
        os.makedirs(current_dir)
    X,y = data_generation()
    rho = float(y.sum()) / len(y)
    if noise:
        X_n = X + np.random.normal(scale=0.1,size=[n, d])
    else:
        X_n = X
    np.savez(os.path.join(current_dir,'data.npz'), X, X_n, y, rho)
    stochastic_scores, tree_scores = run_comparison(X,y)
    np.savez(os.path.join(current_dir,'results.npz'), stochastic_scores, stochastic_scores.mean(), 
             tree_scores, tree_scores.mean(), svm_scores, svm_scores.mean())

In [ ]:
for i in range(5):
    for b in [True,False]
    manage_run(get_simulation_data_1,'sim1-',noise)
for i in range(5):
    manage_run(get_simulation_data_2,'sim2-')
for i in range(5):
    manage_run(get_simulation_data_3,'sim3-')
for i in range(5):
    manage_run(get_simulation_data_4,'sim4-')